# EDA Datos Google Maps / Yelp / Datos Macroeconómicos

## ÍNDICE

1. [YELP](#YELP)

2. [METADATA](#METADATA)

3. [DATOS_MACROECONOMICOS](#DATOS_MACROECONOMICOS)

4. [CALIFORNIA](#CALIFORNIA)

# Introducción

En este análisis exploratorio de datos (EDA), se analizarán las reseñas tanto de Google Maps coomo de Yelp pertenecientes al estado de California para comprender mejor las opiniones y percepciones de los usuarios sobre los establecimientos turísticos y de ocio. Este estudio tiene como objetivo identificar las tendencias y patrones en las calificaciones y comentarios, así como destacar los aspectos más valorados por los clientes y las áreas que requieren mejoras.

Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import ast
import json
import jsonlines
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import matplotlib as plt
import os
import warnings
import gc
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

Carga de datasets

In [3]:
#EDA Datos macroeconómicos
df_macro= pd.read_csv(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\material PF\Yelp\DATOS MACROECONIMICOS USA.csv",encoding='latin-1',sep=";",decimal=",")

#EDA Yelp
data = []
with jsonlines.open(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\material PF\Yelp\review-002.json") as reader:
    for obj in reader:
        data.append(obj)

df_yelp = pd.DataFrame(data)
df_yelp 

#EDA Metadata Google
data = []

with jsonlines.open(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\material PF\Google Maps\metadata-sitios\2.json") as reader:
    for obj in reader:
        data.append(obj)

df_metadata = pd.DataFrame(data)

#EDA Estado de California
df_california = pd.read_parquet(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\reviews.parquet")

In [ ]:
df_macro

In [ ]:
df_yelp

In [ ]:
df_metadata

In [ ]:
df_california

# YELP

Ver valores faltantes y columnas

In [ ]:
df_yelp.info()

Analizar los tipos de datos presentes en cada columna

In [ ]:
df_yelp.isna().sum()

Podemos ver que no hay ningun valor faltante ni valores nulos.

Ver los valores unicos de Stars

In [ ]:
df_yelp['stars'].unique()

# Gráficos

Analizamos algunas muestras

In [ ]:
plt.figure(figsize=(8,6))
df_yelp.boxplot(column=['stars','useful','funny','cool'], sym='ro')
plt.title('Boxplot de reviews')
plt.xlabel('columnas')
plt.ylabel('cantidad')

plt.show()

Existen muchas cantidad de outliers en la columna 'useful', 'funny' y 'cool' . La columna Stars esta con parametros correctos. 
La gran cantidad de valores bajos en el boxplot indica que la mayoría de los usuarios participa de manera moderada, publicando pocas reseñas y recibiendo pocos votos en las diferentes categorías.

Nube de palabras

In [ ]:
# Tomar las primeras 1000 filas de la columna 'text'
df_review_subset = df_yelp.head(10000)

In [ ]:
text = " ".join(review for review in df_review_subset.text)

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
tam_muestra = 1000
muestra = pd.read_json(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\material PF\Google Maps\metadata-sitios\2.json",
lines=True, chunksize=tam_muestra)

valores = []

for valor in muestra:
    
    valores_uso = valor['stars']
    
    valores.extend(valores_uso)


plt.hist(valores, bins=10, edgecolor='black')
plt.xlabel('Valores de stars')
plt.ylabel('Frecuencia')
plt.title('Distribución de la columna "star"')
plt.show()

Se ve una tendencia de mayor frecuencia de puntuaciones por encima de 3.5 en las reseñas.
La mayoría de las experiencias de los usuarios son positivas. Vemos que los usuarios tienden a calificar favorablemente los negocios que están revisando.



In [ ]:
tam_muestra = 10000
muestra = pd.read_json(r'D:\Denise_Estudio\henry\Proyecto final\data\Yelp\review.json',
lines=True, chunksize=tam_muestra)

useful_list = []
stars_list = []


for data_block in muestra:
    useful_list.extend(data_block['useful'])
    stars_list.extend(data_block['stars'])


plt.scatter(stars_list, useful_list, alpha=0.5)
plt.xlabel('Puntaje de Estrellas')
plt.ylabel('Votos ')
plt.title('Puntaje de Estrellas')
plt.show()

En este gráfico queda en evidencia que con las estrellas con valores mas bajos hay mas interaccion,  siendo este review muy calificado como 'useful' por otros usuarios.


# Datasets TIP


Tips (consejos) escritos por el usuario. Los tips son más cortas que las reseñas y tienden a dar sugerencias rápidas.

In [ ]:
data = []

# Leer el archivo JSON Lines
with jsonlines.open(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\material PF\Yelp\tip.json") as reader:
    for obj in reader:
        data.append(obj)

# Convertir la lista de objetos JSON en un DataFrame
df_tips = pd.DataFrame(data)
df_tips

Se revisan algunas columas, tipos de datos y nulos

In [ ]:
df_tips.info()

In [ ]:
df_tips.isna().sum()

In [ ]:
df_tips['compliment_count'].value_counts()

Podemos notar que no hay presencia de valos nulos

# Gráficos

Comencemos a analizar con una nube de palabras sobre una muestra de 10000 filas cuales se repiten más

In [ ]:
muestra_tip = df_tips.head(10000)

In [ ]:
text = " ".join(review for review in muestra_tip.text)

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

Se puede observar que parece existir una tendencia a dejar comentarios buenos y favorables, dado que resaltan palabras como 'great', 'best' ,'good'.

Intentemos con un filtro, por ejemplo donde aparezca la palabra 'food' para conocer las palabras o frases asociadas que más se repiten.

In [ ]:
filtro = df_tips[df_tips['text'].str.contains('food', case=False)]

muestra2_tip = filtro.head(10000)

In [ ]:
text2 = " ".join(review for review in muestra2_tip.text)

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text2)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

Análisis de Comentarios en el Ámbito Gastronómico:

Al aplicar un filtro sobre la muestra de datos analizados, se observa una prevalencia de términos y frases favorables relacionados con el ámbito gastronómico. Palabras y expresiones como "great food", "good", y "great service" son recurrentes en los comentarios, indicando una tendencia predominante hacia evaluaciones positivas. Esto sugiere que, en el dataset analizado (TIPS), los comentarios favorables superan en cantidad a las quejas.

# Dataset check in

Creación del dataframe

In [ ]:
data = []

# Leer el archivo JSON Lines
with jsonlines.open('D:\Denise_Estudio\henry\Proyecto final\data\Yelp\checkin.json') as reader:
    for obj in reader:
        data.append(obj)

df_check = pd.DataFrame(data)
df_check

Revisamos columnas, tipos de datos, nulos 

In [ ]:
df_check.info()

No hay presencia de nulos

Además podemos ver de este dataset que conotiene la informacion referida a cada negocio asociado por su id, y fechas de registros o cambios que se generen dentro de la plataforma, lo cual nos brinda informacion de los registros de cada negocio y en que fecha se realizo.

# Dataset business

Contiene información del comercio, incluyendo localización, atributos y categorías.

Se crea el dataframe

In [ ]:
df_business = pd.read_pickle(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\material PF\Yelp\business.pkl")

Se busca informacion sobre las columnas, variables y valores nulos

In [ ]:
df_business

In [ ]:
df_business.info()

In [ ]:
df_business.isna().sum()

Se observa que hay columnas repetidas, las cuales tienen demasiados nulos, dado que hablamos de gran porcentaje opto por borrarlas ya que no podran brindar informacion útil

In [ ]:
df_business = df_business.loc[:, ~df_business.columns.duplicated()]
df_business

In [ ]:
df_business.info()

Busco valores duplicados revisando id

In [ ]:
duplicates = df_business.duplicated(subset=['business_id'], keep=False)
duplicate_ids = df_business[duplicates]['business_id'].unique()
duplicate_ids

No se encuentran valores duplicados

In [ ]:
df_business.isna().sum()

In [ ]:
df_business['categories'].value_counts()


Analicemos por estados, que valores hay y en cuales hay mayores frecuencias

In [ ]:
unique_states = df_business['state'].nunique()
unique_states

In [ ]:
df_business['state'].value_counts()

Analisis de manera Gráfica

In [ ]:
#
#
#
#
#
#
#warning
#
#
#
#
#

columnas = ["state","stars"]

#Gráficos de barras de conteo
fig, ax = plt.subplots(nrows=len(columnas),ncols =1, figsize=(10,45))
fig.subplots_adjust(hspace=0.5)

for i, col in enumerate(columnas):
    sns.countplot(x=col, data=df_business,ax=ax[i], color='red')
    ax[i].set_title(col)
    ax[i].set_xticklabels(ax[i].get_xticklabels(),rotation=30)

In [ ]:
mayores_negocios = df_business.name.value_counts()

In [ ]:
categoria_counts =df_business['categories'].value_counts()

# Seleccionar los 25 nombres más frecuentes
top_25_categorias = categoria_counts.head(25)

# Graficar los 25 caaategorías más frecuentes
plt.figure(figsize=(12, 8))
sns.barplot(x=top_25_categorias.values, y=top_25_categorias.index, color='red')
plt.xlabel('Conteo de usuarios')
plt.ylabel('Categorías')
plt.title('25 usuarios más Frecuentes')
plt.show()

Se pueden analizar las categorias de negocios que más se repiten, siendo los más repetidos salones de belleza, restaurantes (que difieren en categorias entre ellos) y hoteles.

In [ ]:
mayores_negocios.head(20)

Podemos observar que la matoria de cadenas que se repiten son del ámbito gastronómico, vamos a verlo de manera gráfica.

In [ ]:
business_count = df_business['name'].value_counts()
top_30_categorias = business_count.head(35)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_30_categorias.values, y=top_30_categorias.index, color='Red')
plt.xlabel('Conteo de repeticiones')
plt.ylabel('Cadenas')
plt.title('30 cadenas más Frecuentes')
plt.show()

Podemos notar que en su mayoria existen muchisimos negocios usuarios de Yelp que sse corresponden al rubro gastronomico, viendo aqui los mejores por filtrado de la columna 'name', que corresponden a las mas grandes cadenas existentes en USA.

Filtremos esto para ver quienes tienen mayor frecuencia en un Estado puntual, California.

In [ ]:
df_business_california = df_business.loc[df_business['state'] == 'CA']
df_business_california


In [ ]:
business_count_ca = df_business_california['name'].value_counts()
top_categorias = business_count_ca.head(35)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_categorias.values, y=top_categorias.index, color='Red')
plt.xlabel('Conteo de repeticiones')
plt.ylabel('Marcas')
plt.title('25 cadenas más Frecuentes')
plt.show()

En el Estado de California vemos que existen 30 locales Starsbucks, siendo la cadena con mayor presencia en el estado. Seguido de Mc Donal's y Subway 

Se realiza un merge para saber que establecimientos son:

In [ ]:
filtered_df = pd.merge(df_business_california, top_categorias, on='name', how='inner')
filtered_df

Filtrar negocios con estrellas entre 3 y 5 estrellas  en todo el dataframe de business para saber luego las que no estan tan presentes y tienen mejoes reseñas

high_rating_df: Contiene solo las filas de filtered_df donde la calificación de estrellas está entre 2 y 5.
business_presence: Calcula cuántas veces aparece cada nombre en high_rating_df. Si se corrige el error y se convierte en un DataFrame, mostrará dos columnas: 'name' y 'frecuencia', donde 'frecuencia' es el número de veces que aparece cada nombre.

Filtrado de resultados entre 3 y 5 stars para tener los nombres de negocios con mejores reseñas

In [ ]:
filtered_business = df_business[(df_business['stars'] >= 3) & (df_business['stars'] <= 5)]

In [ ]:
name_count = filtered_business['name'].value_counts()
top_names = name_count.head(35)


Graficar los resultados

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x=top_names.values, y=top_names.index, color='Red')
plt.xlabel('Conteo de repeticiones')
plt.ylabel('Nombres')
plt.title('35 nombres más frecuentes con estrellas entre 3 y 5')
plt.show()



Eje X (horizontal): Representa el conteo de repeticiones, es decir, la cantidad de veces que cada nombre de negocio aparece en las reseñas con calificaciones de entre 3 y 5 estrellas.

Eje Y (vertical): Muestra los nombres de los negocios más frecuentes dentro del rango de estrellas especificado.

Los negocios representados en este gráfico son aquellos que han recibido una cantidad significativa de reseñas con calificaciones de entre 3 y 5 estrellas, lo que puede implicar una satisfacción moderada a alta por parte de los usuarios.
La frecuencia de los nombres sugiere que estos negocios tienen una presencia notable en la base de datos de reseñas, posiblemente debido a su popularidad y amplia distribución.

Conclusión de sección:

Este análisis visual permite identificar rápidamente qué negocios son los más mencionados dentro de las reseñas con calificaciones positivas y moderadas,proporcionando una visión sobre cuáles establecimientos tienen una mayor visibilidad y reciben una cantidad considerable de opiniones por parte de los usuarios.

Relacionando con el gráfico anterior, podemos seleccionar que cadenas presentes en este gráfico son las que tienen mejores reseñas a nivel nacional, para luego identificar en el Estado de California las que tengan oportunidad de crecimiento.


VAMOS A ANALIZARL ALGUNAS

In [ ]:
burger_king_indices = df_business[df_business['name'] == 'Burger King'].index

# Usar iloc para seleccionar la columna 'categories' en esos índices
burger_king_categories = df_business.iloc[burger_king_indices][['name', 'categories']]

# Mostrar los resultados
print(burger_king_categories)

In [ ]:
df_business.iloc[1329]['categories']

In [ ]:
taco_bell_indices = df_business[df_business['name'] == 'Taco Bell'].index

# Usar iloc para seleccionar la columna 'categories' en esos índices
taco_bell_categories = df_business.iloc[taco_bell_indices][['name', 'categories']]

# Mostrar los resultados
print(taco_bell_categories)

In [ ]:
df_business.iloc[1067]['categories']

# Dataset user

Este dataset contiene datos del usuario incluyendo referencias a otros usuarios amigos y a toda la metadata asociada al usuario.

Cargo el dataset

In [ ]:
df_user = pd.read_parquet(r"E:\Repositorios y bases de datos\Henry DS\_PROYECTO FINAL\material PF\Yelp\user-001.parquet")

In [ ]:
df_user

In [ ]:
df_user.info()

Reviso los valores nulos

In [ ]:
df_user.isna().sum()

Este dataset posee multiples columnas con informacion referente a los usuaarios de Yelp, y diferentes promedios. No posee nulos o datos faltantes

Reviso si hay valores repetidos.

In [ ]:
df_user.duplicated().sum()

Existen 117700 filas repetidas, como esto no va a darme informacion fiable, decido eliminarlos para un mejor analisis

In [ ]:
df_user.drop_duplicates(inplace=True)

Reviso los outliers en las columnas numéricas

In [ ]:
plt.figure(figsize=(8,6))
df_user.boxplot(column=['review_count','useful','funny','cool','fans','average_stars'], sym='ro')
plt.title('Boxplot de users')
plt.xlabel('columnas')
plt.ylabel('cantidad numerica')

plt.show()

Podemos ver que existen muchos valores outliers en cuanto a las columnas'useful','funny','cool' . Algunas pocas con datos de reviews y no hay outliers en average_stars, lo cual se corresponde a lo que cuenta la columna.

El gráfico boxplot presentado muestra la distribución de diferentes métricas de interacción de usuarios en un conjunto de datos. Las métricas evaluadas son:

review_count: La cantidad de reseñas realizadas por los usuarios.
useful: La cantidad de votos que indican que una reseña fue útil.
funny: La cantidad de votos que indican que una reseña fue divertida.
cool: La cantidad de votos que indican que una reseña fue genial.
fans: La cantidad de seguidores que tiene cada usuario.
average_stars: La calificación promedio de las reseñas de cada usuario.

La mayoría de los usuarios tienen valores bajos en todas las métricas evaluadas, lo cual se refleja en la concentración de datos cerca del eje horizontal.
Hay presencia de numerosos valores atípicos (outliers) que se extienden hacia valores muy altos en todas las métricas, especialmente en las categorías de 'useful', 'funny' y 'cool'.

Las métricas 'useful', 'funny' y 'cool' muestran una gran cantidad de valores atípicos que alcanzan hasta los 200,000 votos, indicando que algunos usuarios reciben una cantidad excepcionalmente alta de interacciones en sus reseñas.

La alta concentración de valores bajos en el boxplot sugiere que la mayoría de los usuarios tienen una participación moderada, con pocas reseñas y votos en las distintas categorías.

In [ ]:
plt.figure(figsize=(18,10))
df_user.boxplot(column=['compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note','compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos'], sym='ro')
plt.title('Boxplot de users')
plt.xlabel('columnas')
plt.ylabel('cantidad numerica')

plt.show()

El gráfico boxplot muestra la distribución de diferentes tipos de cumplidos recibidos por los usuarios.
 
Las categorías evaluadas son:

1) compliment_more: Más cumplidos.
2) compliment_profile: Cumplidos sobre el perfil.
3) compliment_cute: Cumplidos por ser lindo/a.
4) compliment_list: Cumplidos por listas.
5) compliment_note: Cumplidos por notas.
6) compliment_plain: Cumplidos sencillos.
7) compliment_cool: Cumplidos por ser genial.
8) compliment_funny: Cumplidos por ser gracioso/a.
9) compliment_writer: Cumplidos por ser buen escritor/a.
10) compliment_photos: Cumplidos por fotos.

Existen varios valores atípicos (outliers) que se extienden hacia valores muy altos, particularmente en las categorías de 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', y 'compliment_photos'. Estos valores atípicos resaltan la existencia de usuarios altamente reconocidos o populares, quienes reciben una cantidad significativamente mayor de cumplidos en comparación con el promedio.

Quiero analizar las mejores oportunidades de negocios, por lo que se buscaran franquicias con pocas sucursales y buenas reseñas.

Para esto me dirijo a al dataframe reviews de Yelp.

In [ ]:
df_yelp.info()

In [ ]:
average_stars = df_yelp.groupby('business_id')['stars'].sum()

top_business = average_stars.sort_values(ascending=False).head(30)

top_business

In [ ]:
df_user.info()

# METADATA

# Nulos

In [ ]:
df_metadata.isnull().sum()

In [ ]:
print(df_metadata[df_metadata.isnull().any(axis=1)])

# Vacíos

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
def is_empty(x):
    if x is None:
        return True
    elif isinstance(x, (str, list, dict, set, np.ndarray)) and len(x) == 0:
        return True
    return False

# Aplicar la función al DataFrame
empty_mask = df_metadata.applymap(is_empty)

# Ver el número de valores vacíos en cada columna
print("\nValores vacíos por columna:")
print(empty_mask.sum())

# Ver las filas que contienen valores vacíos
print("\nFilas con valores vacíos:")
print(df_metadata[empty_mask.any(axis=1)])

# Tipos de datos

In [ ]:
#Evaluamos con qué tipo de datos contamos. Nos interesa buscar especialmente cadenas de texto y números enteros
print(df_metadata.dtypes)

# Metadatos

In [ ]:
# Las siguientes celdad muestran datos generales del data frame
df_metadata.info()

In [ ]:
# Imprimir las dimensiones del data frame
print(df_metadata.shape)

In [ ]:
#Visualización de los nombres de las columnas
print(df_metadata.columns)

In [ ]:
# Datos estadísticos
print(df_metadata.describe())      #Se procederá en el ETL a transformar la notación

In [ ]:
unique_misc = df_metadata['gmap_id'].unique()
#no se le puede aplicar a 'MISC', 'category', 'hours', 'relative_results' por ser de tipo lista.

Ejemplo de cómo se estructuran los valores en la columna 'category'

In [ ]:
# Se elije esta columna por ser fundamental para el análisis y por ser muy compleja.
df_metadata.loc[7, 'category']

In [ ]:
# Vista rápida de la frecuencia de cada valor único en la columna 'category'
category_counts = df_metadata['category'].value_counts()
print(category_counts)


# Duplicados

In [ ]:
#Crear una lista para almacenar las columnas que causaron problemas
columnas_problema = []

def check_duplicates(dataframe):
    global columnas_problema
    
    # Crear una lista para almacenar las columnas que se pueden procesar
    columnas_procesables = []

    for col in dataframe.columns:
        try:
            # Intentar acceder a los datos de la columna
            df_metadata[columnas_procesables].append(col)
        except Exception as e:
            columnas_problema.append(col)
            

    # Crear un DataFrame solo con las columnas procesables
    df_procesable = dataframe[columnas_procesables]
    
    # Verificar duplicados en el DataFrame procesable
    try:
        duplicados = df_procesable.duplicated()
        num_duplicados = duplicados.sum()
        if num_duplicados > 0:
            print(f"Número de filas duplicadas en el DataFrame: {num_duplicados}")
            print("Filas duplicadas:")
            print(dataframe[duplicados])
        else:
            print("No hay filas duplicadas en el DataFrame.")
    except Exception as e:
        print(f"Error al verificar duplicados en el DataFrame procesable: {e}")

# Verificar duplicados en el DataFrame
check_duplicates(df_metadata)

# Listar las columnas problemáticas
if columnas_problema:
    print("Columnas que no pueden ser accedidas:")
    for col in columnas_problema:
        print(col)
        

# Outliers

In [ ]:
def calculate_outliers_iqr(column):
    # Eliminar valores nulos antes de calcular
    column = column.dropna()

    # Calcular Q1 (primer cuartil) y Q3 (tercer cuartil)
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1

    # Definir los límites para identificar outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identificar los outliers
    outliers = column[(column < lower_bound) | (column > upper_bound)]

    return outliers

# Aplicar la función a `avg_rating`
avg_rating_outliers = calculate_outliers_iqr(df_metadata['avg_rating'])
print("Outliers en avg_rating:")
print(avg_rating_outliers)

In [ ]:
# Crear el boxplot para visualizar los outliers
plt.figure(figsize=(10, 6))  # Crear una nueva figura con tamaño específico
plt.boxplot(df_metadata['avg_rating'].dropna(), vert=False, patch_artist=True)  # Crear el boxplot
plt.title('Boxplot de avg_rating')  # Título del gráfico
plt.xlabel('avg_rating')  # Etiqueta del eje X
plt.show()  # Mostrar el gráfico

La mayoría de los ítems (probablemente productos o servicios) reciben calificaciones promedio bastante positivas, con la mayor parte entre 3.8 y 4.5.
Hay una cantidad significativa de outliers con calificaciones bajas, lo que podría indicar algunos ítems problemáticos o experiencias negativas consistentes para ciertos productos/servicios.
La asimetría sugiere que es más común ver calificaciones muy bajas que calificaciones perfectas de 5.0.
La concentración de calificaciones en el rango alto podría indicar una tendencia general de satisfacción del usuario o posiblemente un sesgo positivo en el sistema de calificación.

In [ ]:
# Crear histograma para `avg_rating`
plt.figure(figsize=(12, 6))
plt.hist(df_metadata['avg_rating'].dropna(), bins=30, edgecolor='black', alpha=0.7)
plt.title('Histograma de avg_rating')
plt.xlabel('avg_rating')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

La alta concentración de calificaciones de 5 estrellas podría indicar una tendencia de los usuarios a dar calificaciones perfectas con frecuencia, posiblemente inflando las calificaciones generales.
El sesgo hacia calificaciones altas podría deberse a varios factores:

Satisfacción general alta con los productos/servicios.
Tendencia de los usuarios satisfechos a dejar más reseñas que los insatisfechos.
Posible sesgo de selección donde los usuarios compran productos que esperan les gusten.
Posible presencia de reseñas falsas o incentivadas que tienden a ser muy positivas.


La relativamente baja frecuencia de calificaciones medias (2.5-3.5) sugiere que los usuarios tienden a tener opiniones más polarizadas.
El pequeño pico en 1.0 podría representar usuarios muy insatisfechos o productos problemáticos.
La granularidad observada en valores enteros y medios sugiere que muchos usuarios simplifican sus calificaciones, lo que podría afectar la precisión de las calificaciones promedio.

In [ ]:
# Aplicar la función a `num_of_reviews`
num_of_reviews_outliers = calculate_outliers_iqr(df_metadata['num_of_reviews'])
print("Outliers en num_of_reviews:")
print(num_of_reviews_outliers)


In [ ]:
# Crear el boxplot para visualizar los outliers
plt.figure(figsize=(10, 6))  # Crear una nueva figura con tamaño específico
plt.boxplot(df_metadata['num_of_reviews'].dropna(), vert=False, patch_artist=True)  # Crear el boxplot
plt.title('Boxplot de num_of_reviews')  # Título del gráfico
plt.xlabel('num_of_reviews')  # Etiqueta del eje X
plt.show()  # Mostrar el gráfico

La gran mayoría de los elementos (probablemente productos o servicios) tienen muy pocas reseñas. Esto podría indicar que muchos productos son nuevos, poco populares, o que los usuarios no tienden a dejar reseñas con frecuencia.
Hay un pequeño número de elementos "superestrellas" que han acumulado un número excepcionalmente alto de reseñas. Estos podrían ser productos muy populares, best-sellers, o elementos que han estado en el mercado por mucho tiempo.
La distribución sugiere una "economía de los superestrellas" donde unos pocos productos acaparan la mayoría de la atención y las reseñas, mientras que la gran mayoría recibe poca atención.
Esta distribución podría afectar la confiabilidad de las calificaciones promedio, ya que muchos productos tienen muy pocas reseñas para ser estadísticamente significativas.
Para análisis futuros, podría ser útil considerar separadamente los productos con un número significativo de reseñas de aquellos con muy pocas.
Desde una perspectiva de negocio, esto podría indicar oportunidades para fomentar más reseñas en la mayoría de los productos, o para entender qué hace que ciertos productos obtengan tantas reseñas.

In [ ]:
# Crear histograma para `num_of_reviews`
plt.figure(figsize=(12, 6))
plt.hist(df_metadata['num_of_reviews'].dropna(), bins=30, edgecolor='black', alpha=0.7)
plt.title('Histograma de num_of_reviews')
plt.xlabel('num_of_reviews')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

La gran mayoría de los elementos (productos/servicios) tienen muy pocas reseñas, posiblemente incluso cero. Esto podría indicar:
a) Muchos productos nuevos o poco conocidos en el catálogo.
b) Baja participación de los usuarios en dejar reseñas para la mayoría de los productos.
c) Un catálogo muy extenso con muchos productos de nicho.
Solo una pequeña fracción de los elementos recibe un número significativo de reseñas. Estos podrían ser:
a) Productos muy populares o best-sellers.
b) Productos que han estado en el mercado por mucho tiempo.
c) Productos que por alguna razón generan más engagement de los usuarios.
Esta distribución plantea desafíos para la interpretación de las calificaciones promedio:
a) Para la mayoría de los productos, las calificaciones promedio podrían no ser estadísticamente significativas debido al bajo número de reseñas.
b) Los productos con muchas reseñas podrían dominar las métricas generales del sistema.

Sugerencias para el cliente: 
Necesidad de estrategias para incentivar más reseñas en la mayoría de los productos.
Posible revisión de la visibilidad o relevancia de productos con pocas o ninguna reseña.
Considerar ponderaciones o ajustes en los sistemas de recomendación para no favorecer excesivamente a los productos con muchas reseñas.
Investigar qué factores contribuyen a que ciertos productos obtengan muchas más reseñas que otros.

## Conclusión

Hay columnas con una cantidad significativa de valores vacíos (como description, price, hours, MISC, state, relative_results). Se espera manipular estos datos en el proceso de ETL.
Las columnas latitude, longitude, avg_rating, y num_of_reviews parecen estar completas y en su mayoría bien formateadas, lo que es positivo para el análisis.

La mayoría de los negocios tienen calificaciones promedio bastante altas, con la mayoría de las calificaciones entre 3.8 y 4.5. Sin embargo, hay una notable cantidad de outliers con calificaciones bajas y un pequeño pico en 1.0, lo que podría indicar algunos problemas con ciertos negocios o experiencias negativas consistentes.

La distribución de reseñas muestra que la mayoría de los negocios tienen pocas reseñas, lo que sugiere que muchos negocios son nuevos o menos conocidos. Sin embargo, hay un pequeño número de negocios con un número excepcionalmente alto de reseñas.

La alta concentración de calificaciones de 5 estrellas podría ser un indicativo de un sesgo positivo en el sistema de calificaciones. Esto podría ser resultado de una satisfacción general alta, una tendencia de usuarios satisfechos a dejar más reseñas, o incluso reseñas incentivadas.

La baja frecuencia de calificaciones medias sugiere que los usuarios tienden a ser más polarizados en sus opiniones, dando calificaciones muy altas o muy bajas.

La distribución de reseñas sugiere que unos pocos negocios reciben la mayoría de la atención y reseñas, mientras que la mayoría recibe muy poca. Esto puede indicar oportunidades para aumentar la visibilidad de productos menos reseñados o entender por qué ciertos negocios reciben tantas reseñas.

De aquí se deduce que se puede mejorar en:
Desarrollar estrategias para aumentar el número de reseñas en la mayoría de los productos, especialmente en aquellos con pocas o ninguna reseña.
Considerar la visibilidad o relevancia de productos con pocas reseñas para mejorar su presencia.
Ajustar los sistemas de recomendación para no favorecer excesivamente a los negocios con muchas reseñas. Esto podría implicar ponderar las reseñas de manera que reflejen mejor la calidad general del negocio.
Investigar los factores que contribuyen a que ciertos negocios reciban muchas más reseñas que otros, lo cual puede proporcionar insights para mejorar las estrategias de marketing y engagement.



# DATOS_MACROECONOMICOS

In [ ]:
df_macro.head(10)

In [ ]:
df_macro.info()

In [ ]:
df_2023 = df_macro[df_macro['Fecha'] == 2023]

Introducción a la sección:

Al considerar la viabilidad de abrir un negocio o expandirse en un nuevo mercado, es crucial evaluar una serie de variables económicas y demográficas que proporcionan una visión integral del entorno empresarial. Entre las variables que consideramos mas importantes para analizar encontramos la población, la densidad poblacional, el PIB y el PIB per cápita, las cuales son fundamentales para entender el potencial de un mercado y las oportunidades de crecimiento del mismo.

Al evaluar la población podemos llegar a entender el tamaño del mercado, la densidad poblacional nos permitira identificar cómo están distribuidos los consumidores, y el PIB y el PIB per cápita brindaran información sobre la capacidad económica y el poder adquisitivo de los residentes de un determinado lugar. 

Tomar en cuenta estas variables nos permitira tomar una decision informada sobre en cual de lo estados de EE.UU. tenemos que dirigir nuestra lupa para analizar la información obtenida de Yelp y de Google maps con el objetivo de diseñar una estrategias de marketing efectiva, un sistema de recomendaciones y adaptar una oferta a las necesidades y expectativas del mercado objetivo que escojamos.


In [ ]:
top_5_states = df_2023.nlargest(10, 'Poblacion')

# Crear el gráfico de barras con todos los estados
plt.figure(figsize=(10, 6))
bars = plt.bar(df_2023['ESTADO'], df_2023['Poblacion'], color='skyblue')
plt.xlabel('Estado')
plt.ylabel('Población')
plt.title('Población de los Estados en 2023')
plt.xticks(rotation=90)

# Añadir los valores de población encima de las barras solo para los 5 estados con mayor población
for bar, estado in zip(bars, df_2023['ESTADO']):
    if estado in top_5_states['ESTADO'].values:
        yval = bar.get_height()
        yval_formatted = f'{yval:,.0f}'  # Formatear con puntos para separar los miles
        plt.text(bar.get_x() + bar.get_width()/2, yval, yval_formatted, va='bottom', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## Población 

- La población de una región indica el tamaño del mercado potencial disponible para un negocio. Una mayor población sugiere un mercado más grande con más clientes potenciales, lo que puede traducirse en mayores oportunidades de ventas y crecimiento. Por ejemplo, abrir un negocio en una ciudad con una gran población podría atraer a más clientes y generar más ingresos en comparación con una localidad con una población más pequeña.

- En estados con grandes poblaciones como lo son California, Texas, Florida,New York y Pensilvania se tiene una base amplia de clientes potenciales. Esto aumenta la posibilidad de atraer una cantidad significativa de clientes y generar ventas consistentes.

- Más personas en una área significa mayor exposición para el negocio, lo que puede facilitar que el negocio sea conocido y se convierta en un punto de referencia local.

- Las campañas publicitarias pueden beneficiarse dado la mayor concentración de audiencia. La publicidad en medios locales, como televisión, radio, y redes sociales, puede alcanzar a una gran cantidad de personas de manera más efectiva.

- En regiones con alta población, los negocios pueden beneficiarse de economías de escala. La producción y operación en grandes volúmenes pueden reducir los costos unitarios. Esto significa que un negocio puede operar de manera más eficiente y con mayores márgenes de ganancia en áreas con alta densidad poblacional debido al mayor volumen de ventas.

- La densidad poblacional y el tamaño de la población pueden influir en la competencia en el mercado. En áreas con alta población, es posible que haya una mayor concentración de competidores, lo que puede aumentar la competencia pero también validar la demanda de productos o servicios similares. En contraste, en áreas con baja población, la competencia puede ser menor, pero la demanda también podría ser más limitada.


In [ ]:
top_5_states = df_2023.nlargest(10, 'PIB anual M$')

# Crear el gráfico de barras con todos los estados
plt.figure(figsize=(10, 6))
bars = plt.bar(df_2023['ESTADO'], df_2023['PIB anual M$'], color='green')
plt.xlabel('Estado')
plt.ylabel('PIB anual M$')
plt.title('PIB anual de los Estados en 2023')
plt.xticks(rotation=90)

# Añadir los valores de población encima de las barras solo para los 5 estados con mayor población
for bar, estado in zip(bars, df_2023['ESTADO']):
    if estado in top_5_states['ESTADO'].values:
    yval = bar.get_height()
    yval_formatted = f'{yval:,.0f}'  # Formatear con puntos para separar los miles
    plt.text(bar.get_x() + bar.get_width()/2, yval, yval_formatted, va='bottom', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## Producto Interno Bruto

El PIB indica el tamaño y la salud económica de un estado. Un PIB alto sugiere una economía fuerte con una alta capacidad de gasto, lo cual es favorable para negocios que buscan mercados con poder adquisitivo.

### Descripción Grafica 

De acuerdo a la grafica, el estado con mayor PIB en el año 2023 fue California con un PIB de 3.862.171 M$. Seguido muy de lejos por Texas con 2.563.508 M$ y Nueva York con 
2.152.262 M$

## Beneficios de un PIB alto para un negocio

- Los estados con un alto PIB pueden permitirse realizar grandes inversiones en infraestructura, como transporte, tecnología y energía, que a su vez fomentan el crecimiento económico. Este entorno favorable que crean, sirve para el crecimiento de empresas y startups. La disponibilidad de capital y recursos fomenta el emprendimiento y la innovación.

- De igual forma este tipo de estados suelen tener un alto poder adquisitivo entre sus habitantes. Esto significa que los consumidores están dispuestos a gastar más en bienes y servicios, lo que puede traducirse en mayores ventas e ingresos para el negocio.

### Poblacion y PIB

La alta población y el poder adquisitivo en estados como California proporcionan un mercado grande y diversificado para los productos o servicios de un negocio, facilitando su escalabilidad.

La presencia de múltiples sectores económicos en estos estados permite al negocio diversificar sus ofertas y adaptarse a diferentes segmentos del mercado.

In [ ]:
mean_density = df_2023['Densidad'].mean()

# Filtrar los 12 estados con mayor densidad
top_5_states = df_2023.nlargest(12, 'Densidad')

# Crear el gráfico de barras con todos los estados
plt.figure(figsize=(10, 6))
bars = plt.bar(df_2023['ESTADO'], df_2023['Densidad'], color='red')
plt.xlabel('Estado')
plt.ylabel('Densidad')
plt.title('Densidad Poblacional por Estados en 2023')
plt.xticks(rotation=90)

# Añadir la línea de la media de densidades
plt.axhline(y=mean_density, color='blue', linestyle='--', linewidth=2, label=f'Media de Densidad: {mean_density:.2f}')

# Añadir los valores de densidad encima de las barras solo para los 5 estados con mayor densidad
for bar, estado in zip(bars, df_2023['ESTADO']):
    if estado in top_5_states['ESTADO'].values:
        yval = bar.get_height()
        yval_formatted = f'{yval:,.0f}'  # Formatear con puntos para separar los miles
        plt.text(bar.get_x() + bar.get_width() / 2, yval, yval_formatted, va='bottom', ha='center', fontsize=10)

# Añadir la leyenda
plt.legend()

plt.tight_layout()
plt.show()

## Densidad Poblacional
Es la Cantidad de personas que en promedio habitan por kilómetros cuadrados un area determinada. Tomamos esta variable para evalúaR la viabilidad de abrir un nuevo negocio o expandirse en una determinada área dado que: 

## Beneficios de una densidad Poblacional alta para un negocio

- Los estados con alta densidad poblacional tienen un mayor número de personas por área, lo que significa más clientes potenciales en un radio más pequeño. Esto puede traducirse en mayores ventas y una base de clientes más estable.

- Una mayor concentración de personas generalmente se traduce en una mayor demanda de productos y servicios, lo que puede llevar a un flujo constante de clientes y mayores ingresos.

- En lugares con alta densidad,los costos fijos, como el alquiler y los servicios, pueden distribuirse entre más clientes, lo que mejora la rentabilidad.

- En áreas densamente pobladas, el Marketing digital con sus respectivas estrategias de comunicación por redes puede ser más eficiente porque un anuncio puede llegar a más personas en un área más pequeña, reduciendo los costos por impresión o impacto

### Descripción Grafica 

- Dentro de la grafica podemos visulizar que los Estados con mayor densidad Poblacional son Nueva Jersey,Rhode Island,Connecticut, Massachusetts y Maryland

- La media de la densidad de todos los estados es de 67.55 personas por kilometro cuadrado

- De los estados con mayor Población y PIB encontramos por encima de la media de densidad a New York, Florida, California e Illinois.

### Densidad Poblacional y PIB

la combinación de un alto PIB y una alta densidad poblacional puede crear un entorno económico dinámico y próspero, ofreciendo un mercado vasto y diverso para negocios de todos los sectores. Por esta razon estados como New York, Florida, California e Illinois toman mas fuerza a la hora de decidir cual escoger para nuestro analisis. 


In [ ]:
top_5_states = df_2023.nlargest(5, 'PIB Per Capita $')

# Crear el gráfico de barras con todos los estados
plt.figure(figsize=(10, 6))
bars = plt.bar(df_2023['ESTADO'], df_2023['PIB Per Capita $'], color='black')
plt.xlabel('Estado')
plt.ylabel('PIB Per Capita $')
plt.title('PIB Per Capita por Estados en 2023')
plt.xticks(rotation=90)

# Añadir los valores de población encima de las barras solo para los 5 estados con mayor población
for bar, estado in zip(bars, df_2023['ESTADO']):
    if estado in top_5_states['ESTADO'].values:
    yval = bar.get_height()
    yval_formatted = f'{yval:,.0f}'  # Formatear con puntos para separar los miles
    plt.text(bar.get_x() + bar.get_width()/2, yval, yval_formatted, va='bottom', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

## PIB per capita

El PIB per cápita proporciona una visión de la renta media de los residentes de un
Estado. Un PIB per cápita alto indica un alto nivel de ingresos individuales o lo que se conoce como mayor poder adquisitivo. 

Tomamos esta variable en nuestro analisis para poder identificar los estados que tengan mercados mas desarrollados capaces de soportar negocios que ofrezcan productos o servicios sin importar si son o no de lujo o premium 

En la grafica podemos observar que los estados con mayor PIB per capita son: New York, Massachusetts, Washington y California. Esto puede llegar a indicar que en estos estados dados los altos ingresos de sus habitantes,existe un mayor consumo de bienes y servicios.

## Beneficios de una PIB per capita alto para un negocio

- Una alta capacidad adquisitiva permite a un negocio diversificar su oferta de productos y servicios para satisfacer diferentes segmentos del mercado.

- Un PIB per cápita alto suele indicar una economía estable y en crecimiento, lo que proporciona un entorno empresarial más predecible y seguro.



In [ ]:
# Lista de estados de interés
estados_interes = ['California', 'Nueva York', 'Florida', 'Illinois', 'Massachusetts', 'Texas']
# Filtrar los datos para incluir solo los estados de interés
df_filtrado = df_macro[df_macro['ESTADO'].isin(estados_interes)]

# Crear la gráfica de tendencia
plt.figure(figsize=(12, 8))

# Graficar cada estado por separado
for estado in estados_interes:
    df_estado = df_filtrado[df_filtrado['ESTADO'] == estado]
    plt.plot(df_estado['Fecha'], df_estado['PIB anual M$'], marker='o', label=estado)
    
    # Mostrar los valores en cada punto
    for x, y in zip(df_estado['Fecha'], df_estado['PIB anual M$']):
        plt.text(x, y, f'{y:,.0f}', fontsize=9, ha='right')

plt.xlabel('Año')
plt.ylabel('PIB anual M$')
plt.title('Tendencia del PIB Anual (2019-2023)')
plt.legend()
plt.grid(True)

# Asegurarse de que los años se muestren sin decimales y evitar duplicaciones
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}'))
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))

plt.tight_layout()
plt.show()

Conclusión de sección

# ¿Por qué California?


California, siendo el estado con la mayor población y el PIB más alto de los 50 estados de EE.UU., representa una de las economías más grandes y dinámicas a nivel nacional. Su perfil económico y demográfico ofrece una oportunidad única para la creación o expansión de negocios.

- Cuenta con una población considerablemente grande, lo cual se traduce en un mercado amplio con una base de consumidores extensa. Este gran número de habitantes significa una mayor demanda potencial para una variedad de productos y servicios. Además, una mayor población implica una diversidad de segmentos de mercado, lo cual puede permitir a las empresas adaptar sus ofertas a diferentes necesidades y preferencias.

- El elevado PIB de California indica que el estado tiene una economía robusta y de alto rendimiento. Esto sugiere un ambiente empresarial favorable, con un alto poder adquisitivo entre los residentes. Un PIB alto generalmente está asociado con una mayor capacidad de gasto y una infraestructura económica avanzada, lo que facilita la operación y el crecimiento de nuevos negocios.

- El alto PIB per cápita en California refleja una población con un ingreso medio superior. Esto es crucial para las empresas que buscan ingresar a un mercado donde los consumidores tienen la capacidad financiera para adquirir productos y servicios premium. Un alto PIB per cápita indica un mercado con un mayor poder adquisitivo, lo cual es atractivo para negocios que ofrecen bienes y servicios de alta gama.


- California también se destaca por su alta densidad poblacional en ciertas áreas, como Los Ángeles y San Francisco. Esto significa que hay una concentración significativa de personas en zonas específicas, lo que puede resultar en una mayor visibilidad y accesibilidad para los negocios. Una alta densidad poblacional puede facilitar la captación de clientes y maximizar la eficiencia de las estrategias de marketing al enfocarse en áreas de alta concentración de consumidores.

# CALIFORNIA

In [ ]:
df_california

# Nulos

In [ ]:
df_california.isnull().sum()

In [ ]:
print(df_california[df_california.isnull().any(axis=1)])

## Vacíos

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
def is_empty(x):
    if x is None:
        return True
    elif isinstance(x, (str, list, dict, set, np.ndarray)) and len(x) == 0:
        return True
    return False

# Aplicar la función al DataFrame
empty_mask = df_california.applymap(is_empty)

# Ver el número de valores vacíos en cada columna
print("\nValores vacíos por columna:")
print(empty_mask.sum())

# Ver las filas que contienen valores vacíos
print("\nFilas con valores vacíos:")
print(df_california[empty_mask.any(axis=1)])

# Tipos de datos

In [ ]:
#Evaluamos con qué tipo de datos contamos. Nos interesa buscar especialmente cadenas de texto y números enteros
print(df_california.dtypes)

# Metadatos

In [ ]:
#Las siguientes celdas muestras información general del dataset
df_california.info()

In [ ]:
#imprimir las dimensiones del data frame
print(df_california.shape)

In [ ]:
#Visualización de los nombres de las columnas
print(df_california.columns)

In [ ]:
# Datos estadísticos
print(df_california.describe())       #Se procederá en el ETL a transformar la notación

In [ ]:
#Muestra de valores únicos de ID de usuarios
unique_misc = df_california['user_id'].unique()
print(len(unique_misc))


In [ ]:
#Muestra de valores únicos de ID de locación
unique_misc = df_california['gmap_id'].unique()
print(len(unique_misc))

# Duplicados

In [ ]:
#Crear una lista para almacenar las columnas que causaron problemas
columnas_problema = []

def check_duplicates(dataframe):
    global columnas_problema
    
    # Crear una lista para almacenar las columnas que se pueden procesar
    columnas_procesables = []

    for col in dataframe.columns:
        try:
            # Intentar acceder a los datos de la columna
            df_california[columnas_procesables].append(col)
        except Exception as e:
            columnas_problema.append(col)
            

    # Crear un DataFrame solo con las columnas procesables
    df_procesable = dataframe[columnas_procesables]
    
    # Verificar duplicados en el DataFrame procesable
    try:
        duplicados = df_procesable.duplicated()
        num_duplicados = duplicados.sum()
        if num_duplicados > 0:
            print(f"Número de filas duplicadas en el DataFrame: {num_duplicados}")
            print("Filas duplicadas:")
            print(dataframe[duplicados])
        else:
            print("No hay filas duplicadas en el DataFrame.")
    except Exception as e:
        print(f"Error al verificar duplicados en el DataFrame procesable: {e}")

# Verificar duplicados en el DataFrame
check_duplicates(df_california)

# Listar las columnas problemáticas
if columnas_problema:
    print("Columnas que no pueden ser accedidas:")
    for col in columnas_problema:
        print(col)
        

# Outliers

In [ ]:
# Función para calcular outliers usando el rango intercuartílico (IQR)
def calculate_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return series[(series < lower_bound) | (series > upper_bound)]

# Aplicar la función a la columna `rating`
rating_outliers = calculate_outliers_iqr(df_california['rating'])
print("Outliers en rating:")
print(rating_outliers)

In [ ]:
# Aplicar la función a la columna `time`
time_outliers = calculate_outliers_iqr(df_california['time'])
print("Outliers en time:")
print(time_outliers)


In [ ]:
# Boxplot para la columna 'rating'
plt.subplot(1, 2, 1)
sns.boxplot(data=df_california, x='rating')
plt.title('Boxplot de Rating')

# Boxplot para la columna 'time'
plt.subplot(1, 2, 2)
sns.boxplot(data=df_california, x='time')
plt.title('Boxplot de Time')

# Mostrar los gráficos
plt.tight_layout()
plt.show()

Boxplot de Rating:

La escala va de 1 a 5, lo que sugiere que podría ser una escala de calificación típica.
La mediana (línea horizontal dentro de la caja) está cerca de 4.
La caja, que representa el rango intercuartílico (IQR), se extiende aproximadamente de 3.5 a 5.
Hay algunos valores atípicos (outliers) en el extremo inferior, representados por círculos cerca de 1 y 2.
La distribución parece estar sesgada hacia los valores más altos, ya que la caja está más comprimida en la parte superior.


Boxplot de Time:

La mediana está cerca de 1.4.
La caja es muy estrecha, lo que indica que el 50% central de los datos está muy concentrado.
Hay una gran cantidad de valores atípicos en el extremo inferior, representados por múltiples círculos superpuestos.
La distribución parece estar muy sesgada, con la mayoría de los datos concentrados en valores más altos y una "cola" larga hacia los valores más bajos.

Las calificaciones tienden a ser altas, con la mayoría entre 3.5 y 5, pero con algunos valores bajos atípicos.
Los tiempos están muy concentrados alrededor de un valor alto, pero hay muchos casos atípicos con tiempos significativamente menores.

# Distribución del rating

In [ ]:
# Crear una figura con dos subgráficas
plt.figure(figsize=(14, 5))

# Histograma y gráfico de densidad para la columna 'rating'
plt.subplot(1, 2, 1)
sns.histplot(df_california['rating'], kde=True, bins=20)
plt.title('Distribución de Rating')
plt.xlabel('Rating')
plt.ylabel('Frecuencia')


# Mostrar los gráficos
plt.tight_layout()
plt.show()


Hay dos picos principales, uno en 5.0 y otro menor en 4.0, lo que sugiere una distribución bimodal.
El rating de 5.0 tiene la frecuencia más alta, superando los 2 millones de ocurrencias.
Hay un pico secundario en 4.0, con alrededor de 0.5 millones de ocurrencias.
Hay frecuencias relativamente bajas para los ratings de 1.0, 2.0 y 3.0, aunque 1.0 tiene una frecuencia ligeramente mayor que 2.0 y 3.0.
La distribución está fuertemente sesgada hacia los ratings altos (4 y 5).
Hay muy pocas calificaciones entre los valores enteros, lo que sugiere que los usuarios tienden a dar calificaciones enteras.
La gran diferencia entre las frecuencias de 5.0 y las demás sugiere una tendencia a dar calificaciones extremadamente positivas. (Polarización)

Esta distribución es típica en sistemas de calificación online, donde los usuarios tienden a dar calificaciones muy altas o muy bajas, con menos calificaciones intermedias.

# Nube de palabras de reseñas

In [ ]:
# Función para limpiar el texto
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar puntuación
    words = word_tokenize(text)  # Tokenizar
    stop_words = set(stopwords.words('english'))  # Stopwords en inglés
    words = [word for word in words if word not in stop_words]  # Eliminar stopwords
    return ' '.join(words)

# Unir todas las reseñas en una sola cadena de texto
all_text = ' '.join(df_california['text'].dropna().apply(clean_text))

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

# Mostrar la nube de palabras
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Ocultar los ejes
plt.title('Nube de Palabras de Reseñas')
plt.show()

Palabras más prominentes:
"great" (grande, excelente): Es la palabra más destacada, apareciendo en varios tamaños y colores.
"food" (comida): Muy prominente, sugiriendo que las reseñas son mayormente sobre restaurantes.
"service" (servicio): También muy destacada, indicando la importancia de la atención al cliente.
"place" (lugar): Bastante visible, posiblemente refiriéndose al establecimiento en general.
"google": Notablemente grande, ya que se trata de reseñas de Google Maps.

Otras palabras frecuentes:
"friendly" (amigable)
"staff" (personal)
"good" (bueno)
"recommend" (recomendar)
"translated" (traducido): Interesante, podría indicar que algunas reseñas fueron traducidas automáticamente.
"customer" (cliente)
"price" (precio)

Sentimientos expresados:
Predominantemente positivos: "love", "excellent", "best", "helpful", "nice".
Algunas palabras neutras: "first time", "back" (posiblemente "come back" o "went back").
No se observan palabras negativas prominentes.

Aspectos del servicio mencionados:
"staff" y "friendly" sugieren un enfoque en el trato del personal.
"price" indica que el costo es un factor importante.
"experience" sugiere que los clientes evalúan la experiencia general.

Frecuencia y tiempo:
"always" y "every time" indican consistencia en la experiencia.
"first time" sugiere que hay reseñas de clientes nuevos.

Acción y recomendación:
"recommend" y "highly recommend" son prominentes, indicando que muchos clientes recomiendan el lugar o servicio.
"coming back" sugiere la intención de regresar, lo cual es positivo.

Esta nube de palabras sugiere que las reseñas son generalmente muy positivas, con un fuerte énfasis en la calidad de la comida, el servicio y la experiencia general. 

# Número de reseñas por usuario

In [ ]:
# Convertir el tiempo en formato de fecha
df_california['date'] = pd.to_datetime(df['time'], unit='ms')

# Extraer el año
df_california['year'] = df_california['date'].dt.year

# Contar el número de reseñas por usuario y seleccionar los 20 principales
review_counts = df_california['user_id'].value_counts().head(20)

# Filtrar el DataFrame para solo los 20 usuarios principales
top_users = review_counts.index.tolist()
df_top_users = df_california[df_california['user_id'].isin(top_users)]

# Calcular el número de reseñas por usuario por año
review_counts_by_year = df_top_users.groupby(['user_id', 'year']).size().reset_index(name='counts')

# Ordenar los años
review_counts_by_year = review_counts_by_year.sort_values(by='year')

# Calcular el promedio de reseñas por año
average_reviews_per_year = review_counts_by_year.groupby('year')['counts'].mean().reset_index(name='average_counts')

# Graficar
plt.figure(figsize=(14, 8))

# Graficar las reseñas por usuario
for idx, user_id in enumerate(top_users):
    user_data = review_counts_by_year[review_counts_by_year['user_id'] == user_id]
    plt.plot(user_data['year'], user_data['counts'], marker='o', label=f'Usuario {idx + 1}')

# Graficar la línea de promedio
plt.plot(average_reviews_per_year['year'], average_reviews_per_year['average_counts'], color='black', linestyle='--', linewidth=6, label='Promedio Anual')

# Configurar el gráfico
plt.title('Evolución del Número de Reseñas por Usuario (Top 20) a lo Largo del Tiempo (Anual)')
plt.xlabel('Año')
plt.ylabel('Número de Reseñas')
plt.xticks(rotation=45)
plt.legend(title='Usuarios', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

Hay una gran variabilidad entre usuarios y años, pero se observa un pico notable en 2016 seguido de una tendencia decreciente.
Pico destacado: El Usuario 1 muestra un pico extraordinario en 2016 con más de 250 reseñas, muy por encima del resto. (Outlier a evaluar)
En 2010, varios usuarios (especialmente 1, 3 y 14) muestran una actividad significativa que luego disminuye.
Entre 2012 y 2014, la actividad es generalmente baja para la mayoría de los usuarios.
A partir de 2016, se observa un aumento general en la actividad de reseñas para varios usuarios.
Promedio anual: La línea negra gruesa representa el promedio anual, mostrando una tendencia general que refleja los patrones de actividad colectivos.
Tendencia reciente: Hacia 2020-2022, la actividad parece estabilizarse en niveles más bajos para la mayoría de los usuarios.
Variabilidad individual: Algunos usuarios muestran patrones muy irregulares, con picos aislados en ciertos años.

En resumen, el gráfico muestra una dinámica compleja de la actividad de reseñas, con períodos de alta variabilidad y cambios significativos en los patrones de participación de los usuarios a lo largo del tiempo.

# Promedio de calificaciones por mes

In [ ]:
# Convertir el tiempo en formato de fecha
df_california['date'] = pd.to_datetime(df_california['time'], unit='ms')

# Extraer año y mes
df_california['year_month'] = df_california['date'].dt.to_period('M')

# Calcular el promedio de calificaciones por mes
monthly_ratings = df.groupby('year_month')['rating'].mean().reset_index()

# Convertir el 'year_month' a formato datetime para usar en el gráfico
monthly_ratings['year_month'] = monthly_ratings['year_month'].dt.to_timestamp()

# Graficar
plt.figure(figsize=(12, 6))
plt.scatter(monthly_ratings['year_month'], monthly_ratings['rating'])
plt.title('Promedio de Calificaciones por Mes')
plt.xlabel('Mes')
plt.ylabel('Promedio de Calificaciones')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

A largo plazo, se observa una ligera tendencia al alza en las calificaciones promedio.
Hay una gran variabilidad en los datos, especialmente en el período entre 2000 y 2010, donde se ven fluctuaciones extremas.

Inicio de los 90s hasta finales de los 90s: Aumento constante y suave.
Alrededor del 2000: Caída brusca seguida de alta volatilidad.
2010 en adelante: Estabilización relativa con un aumento gradual.

Los promedios parecen oscilar principalmente entre 3.0 y 5.0.
Se observan algunos picos notables que llegan a 5.0, así como caídas pronunciadas, con el punto más bajo cerca de 2.75.
En los últimos años, parece haber una tendencia más estable y ligeramente ascendente.
Hay algunos puntos de datos que parecen ser valores atípicos, especialmente durante el período de alta volatilidad.

Este gráfico sugiere que ha habido cambios significativos en el sistema de calificación o en los factores que afectan las calificaciones a lo largo del tiempo.

 "- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - "

La mayoría de las calificaciones se agrupan en los valores extremos, con una clara predominancia de calificaciones de 5.0 (más de 2 millones de ocurrencias) y un pico secundario en 4.0 (alrededor de 0.5 millones).
Las calificaciones intermedias (1.0, 2.0 y 3.0) son menos frecuentes, con una notable inclinación hacia calificaciones muy altas (4.0 y 5.0). Esto sugiere que los usuarios tienden a dar calificaciones extremadamente positivas o muy negativas.

A lo largo del tiempo, se observa una ligera tendencia al alza en las calificaciones promedio, a pesar de la alta variabilidad y algunos picos y caídas pronunciadas, especialmente entre 2000 y 2010.
Esto podría indicar una mejora en la calidad del servicio o una mayor satisfacción de los usuarios en años recientes.

Existe un usuario altamente activo (Usuario 1) que contribuye con más de 300 reseñas, lo que representa una proporción significativa del total de reseñas.
Los siguientes usuarios más activos tienen una disminución pronunciada en el número de reseñas, con el Usuario 2 aportando alrededor de 140 reseñas y los siguientes 3-5 usuarios con entre 120-130 reseñas cada uno.
A partir del Usuario 6, la actividad se distribuye más uniformemente, aunque sigue existiendo una brecha significativa entre el Usuario 1 y el resto.

La distribución de reseñas sugiere que un pequeño grupo de usuarios contribuye de manera desproporcionada al número total de reseñas, mientras que la mayoría de los usuarios contribuye de manera menos frecuente.

En cuanto al análisis de sentimientos, las palabras más frecuentes en las reseñas son altamente positivas, con términos como "great", "food", "service" y "recommend" dominando el contenido.
Los sentimientos expresados son principalmente positivos, con pocas palabras negativas prominentes.

Las reseñas destacan la calidad de la comida y el servicio, con palabras como "friendly", "staff", "price" y "experience" apareciendo con frecuencia.
La mayoría de las reseñas parecen centrarse en aspectos positivos de la experiencia, con un enfoque en la calidad del servicio y la comida, y una tendencia a recomendar el lugar.

La variabilidad en las calificaciones y en el número de reseñas a lo largo del tiempo sugiere cambios en la percepción de los usuarios o en la calidad del servicio.
Los datos indican una estabilización en las calificaciones en años recientes después de un período de alta volatilidad, con una ligera tendencia ascendente.

Se observan picos notables y caídas en las calificaciones, lo que podría reflejar cambios en la oferta de servicios, eventos significativos o actualizaciones en el sistema de calificación.

# Conclusión general:
Calidad General: Las reseñas tomadas de California son en su mayoría muy positivas, con una fuerte inclinación hacia calificaciones de 4.0 y 5.0.
Contribución desproporcionada: Un número relativamente pequeño de usuarios contribuye con la mayoría de las reseñas, mientras que la mayoría de los usuarios participa menos activamente.
Tendencias Temporales: Hay una ligera tendencia al alza en las calificaciones promedio a lo largo del tiempo, con una gran variabilidad en los datos.
Se espera que estos hallazgos puedan ser útiles para nuestro cliente para mejorar estrategias de marketing, optimizar la oferta de servicios y comprender mejor las expectativas y comportamientos de los usuarios en California.